In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, auc
import pandas as pd
import numpy as np
from keras.datasets import boston_housing
from keras.datasets import reuters
import numpy as np
import os
import tensorflow as tf
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics
import matplotlib.pyplot as plt
import time
import pandas as pd
import random
from sklearn.utils import shuffle
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.ensemble import *
from sklearn.neighbors import *
from sklearn.datasets import make_classification
import xgboost as xgb
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OutputCodeClassifier
import warnings

Using TensorFlow backend.


In [3]:
warnings.filterwarnings('ignore')
df = pd.read_excel(r'C:\Users\Big data\Desktop\class\funcardproject\斷詞與和卡額度_20群.xls',encoding='utf-16')
df = df.loc[:, ["age","serveTime","credLimit","Loan","SalPerY","holdCard","Career","credLimit_group"]] 
df =df.dropna(
    axis=0,     # 0: 对行进行操作; 1: 对列进行操作
    how='any'   # 'any': 只要存在 NaN 就 drop 掉; 'all': 必须全部是 NaN 才 drop 
    )
df["credLimit_group"] = df["credLimit_group"].astype(int)
print(df.shape)
print(df.info())

(1680, 8)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1680 entries, 0 to 1686
Data columns (total 8 columns):
age                1680 non-null float64
serveTime          1680 non-null float64
credLimit          1680 non-null int64
Loan               1680 non-null float64
SalPerY            1680 non-null int64
holdCard           1680 non-null int64
Career             1680 non-null int64
credLimit_group    1680 non-null int32
dtypes: float64(3), int32(1), int64(4)
memory usage: 111.6 KB
None


In [4]:
model_xgb = xgb.XGBClassifier()
model_GBM = GradientBoostingClassifier()
model_AdaBoost = AdaBoostClassifier()
model_Bagging = BaggingClassifier()
model_ExtraTreesClassifier = ExtraTreesClassifier()
model_SVM = SVC()
model_KNN = KNeighborsClassifier()
model_LDA = LinearDiscriminantAnalysis()
model_rf = RandomForestClassifier()
model_Gaussian = GaussianProcessClassifier()
model_BernoulliNB = BernoulliNB()
model_DecisionTreeClassifier = DecisionTreeClassifier()
model_ExtraTreeClassifier = ExtraTreeClassifier()
model_MultiOutputClassifier = MultiOutputClassifier(SVC())#不能投票,可能是*粉
model_OneVsOneClassifier = OneVsOneClassifier(SVC())
model_OneVsRestClassifier = OneVsRestClassifier(SVC())#不能投票,可能是*粉
model_OutputCodeClassifier = OutputCodeClassifier(SVC())
#清單
modellist=[model_xgb,
           model_GBM,
           model_AdaBoost,
           model_Bagging,
           model_ExtraTreesClassifier,
           model_SVM,
           model_KNN,
           model_LDA,
           model_rf,
           model_Gaussian,
           model_BernoulliNB,
           model_DecisionTreeClassifier,
           model_ExtraTreeClassifier,
           model_MultiOutputClassifier,
           model_OneVsOneClassifier,
           model_OneVsRestClassifier,
           model_OutputCodeClassifier]

In [5]:
def cut(random):
    # 切分訓練,測試
    global train_data, test_data, train_targets, test_targets
    train_data, test_data, train_targets, test_targets = train_test_split(df.loc[:, ["age","serveTime","Loan","SalPerY","holdCard","Career"]] , df.loc[:, ["credLimit_group"]] , test_size=0.3,random_state=random)
    # 標準化
    mean = train_data.mean(axis=0)
    train_data -=mean
    std = train_data.std(axis=0)
    train_data/=std
    test_data-=mean
    test_data/=std
    return train_data, test_data, train_targets, test_targets

In [6]:
def model_test(train_data, test_data, train_targets, test_targets,model):
    model.fit(train_data, train_targets)
    訓練準確度 = model.score(train_data, train_targets)
    驗證準確度 = model.score(test_data, test_targets)
    print(model,"模型")
    print("訓練準確度:",訓練準確度,"%")
    print("訓練準確度:",驗證準確度,"%")
    print("_______________________________________________________________________________")
    

In [7]:
#model_MultiOutputClassifier,('model_OneVsRestClassifier' ),不能用
#可逐一剔除,觀察準確度變化
def vote(train_data, test_data, train_targets, test_targets,i):
    Voting = VotingClassifier(estimators=[('model_xgb', model_xgb),#good
#                                           ('model_GBM', model_GBM),#good
                                          ('model_AdaBoost', model_AdaBoost),
                                          ('model_ExtraTreesClassifier', model_ExtraTreesClassifier),
                                          ('model_SVM', model_SVM),
                                          ('model_KNN', model_KNN),#poor
#                                           ('model_LDA', model_LDA),
                                          ('model_rf', model_rf),
                                          ('model_Gaussian', model_Gaussian),#poor
#                                           ('model_BernoulliNB', model_BernoulliNB),#真的白努力
                                          ('model_DecisionTreeClassifier', model_DecisionTreeClassifier),
                                          ('model_ExtraTreeClassifier', model_ExtraTreeClassifier),
                                          ('model_OneVsOneClassifier', model_OneVsOneClassifier),
                                          ('model_OutputCodeClassifier', model_OutputCodeClassifier)
                                         ])
    Voting.fit(train_data, train_targets)
    訓練準確度 = Voting.score(train_data, train_targets)
    驗證準確度 = Voting.score(test_data, test_targets)
    train = print("訓練準確度:",訓練準確度,"%")
    test = print("測試準確度:",驗證準確度,"%")
    return train,test
cut(4)
vote(train_data, test_data, train_targets, test_targets,4)

訓練準確度: 0.7312925170068028 %
測試準確度: 0.4126984126984127 %


(None, None)

# 測試小綠同學

In [8]:
小綠 = pd.DataFrame(columns=["age","serveTime","Loan","SalPerY","holdCard","Career"])
小綠.loc[0]=8,120,4,600000,1,1
小綠.loc[1]=28,12,0,600000,0,0
小綠.loc[2]=28,12,0,87,2,0
小綠

,age,serveTime,Loan,SalPerY,holdCard,Career
0,8,120,4,600000,1,1
1,28,12,0,600000,0,0
2,28,12,0,87,2,0


In [9]:
# 因為先前輸入時有先標準化,因此輸入也要標準化
小綠-=mean
小綠/=std
小綠

In [11]:
# 他給出的是每一群的機率(相加為一)
preds = Voting.predict(小綠)
preds[0]

NameError: name 'Voting' is not defined

In [ ]:
#接著我們找出裡面機率最大的值的所在位子
print(np.where(preds[0]==np.max(preds[0])))

In [ ]:
preds[1]